In [2]:
import os
import re
import pandas as pd
import numpy as np
import tensorflow as tf

np.random.seed(1)
tf.random.set_seed(2)

import pandas as pd
import keras
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, classification_report, log_loss

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional, Flatten
from keras.layers import Dropout, Conv1D, GlobalMaxPool1D, GRU, GlobalAvgPool1D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

print(os.listdir('../LSTM'))

['model_1b.h5', 'model_plot3.png', 'model_1c.h5', 'text.csv', 'Sentiment.csv', 'LSTM-B-OnlyText.ipynb', 'LearnPipeline.ipynb', 'model_1.h5', 'glove.840B.300d.txt', 'PA-all.csv', 'model_2.h5', 'LSTM-B.ipynb', 'dfout.csv', 'model.hdf5', 'timezone-PA-B.csv', 'LSTM-MultipleFeatures-CleanLab.ipynb', 'glove.twitter.27B.200d.txt', '.ipynb_checkpoints', 'LSTM-MultipleFeatures.ipynb', 'LSTM-B-AllFeatures.ipynb', 'PA-Truth.csv', 'model_1d.h5', 'LSTM-MultipleFeatures-NoV.ipynb', 'PA-MV-LSTM.csv']


In [3]:
data = pd.read_csv('PA-MV-LSTM.csv')
data= data[data['hour']!='dummy']
data['A1']=data['A1'].astype("category")
data['A2']=data['A2'].astype("category")
data['A3']=data['A3'].astype("category")
data['sex']=data['sex'].astype("category")
data['age']=data['age'].astype("category")
data['org']=data['org'].astype("category")
data['weekday']=data['weekday'].astype("category")
data['result']=data['result'].astype("category")
data['hour']=data['hour'].astype("int")
data['month']=data['month'].astype("int")
data['truth']=data['truth'].astype("category")
data.head()



#data['truth']=np.where(data['truth'] ==1, 'YY', data['truth'])
data['truth']=np.where(data['truth'] ==2, '0', data['truth'])
data['truth']=np.where(data['truth'] =='3', '0', data['truth'])
data['truth']=np.where(data['truth'] =='4', '0', data['truth'])
data['truth']=np.where(data['truth'] =='5', '0', data['truth'])

#data['A1']=np.where(data['A1'] ==1, 'YY', data['A1'])
data['A1']=np.where(data['A1'] ==2, '0', data['A1'])
data['A1']=np.where(data['A1'] =='3', '0', data['A1'])
data['A1']=np.where(data['A1'] =='4', '0', data['A1'])
data['A1']=np.where(data['A1'] =='5', '0', data['A1'])
#data['A2']=np.where(data['A2'] ==1, 'YY', data['A2'])
data['A2']=np.where(data['A2'] ==2, '0', data['A2'])
data['A2']=np.where(data['A2'] =='3', '0', data['A2'])
data['A2']=np.where(data['A2'] =='4', '0', data['A2'])
data['A2']=np.where(data['A2'] =='5', '0', data['A2'])
#data['A3']=np.where(data['A3'] ==1, 'YY', data['A3'])
data['A3']=np.where(data['A3'] ==2, '0', data['A3'])
data['A3']=np.where(data['A3'] =='3', '0', data['A3'])
data['A3']=np.where(data['A3'] =='4', '0', data['A3'])
data['A3']=np.where(data['A3'] =='5', '0', data['A3'])

data = pd.get_dummies(data, prefix=['A1', 'A2', 'A3',  'age', 'sex', 'org', 'weekday', 'result'], columns=['A1', 'A2', 'A3', 'age', 'sex', 'org','weekday', 'result'], drop_first=True)
data.head()

data.head()


,Unnamed: 0,iid,hour,text,month,truth,A1_1,A2_1,A3_1,age_30-39,...,age_>=40,sex_male,org_non-org,weekday_Mon,weekday_Sat,weekday_Sun,weekday_Thu,weekday_Tue,weekday_Wed,result_1
0,0,5ece812df46b2b34fcc352ac,22,@murphdog47 Sad to see hockey hat guy go but y...,2,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,0
1,1,5ece812ef46b2b34fcc3a0be,22,You’re very welcome! Thank you for bringing ho...,2,0,0,0,0,0,...,0,1,1,0,0,1,0,0,0,0
2,2,5ece814af46b2b34fcc9528a,21,Random #productivity hacks continued:\n\n🎽 On ...,2,0,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0
3,3,5ece815ef46b2b34fccd1ae2,22,Now they’ll say a few useless scripted platitu...,2,0,0,0,0,0,...,1,1,1,0,1,0,0,0,0,0
4,4,5ece8169f46b2b34fccf47ef,22,A simple exercise you must do everyday to keep...,2,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0


In [4]:
data.columns

Index(['Unnamed: 0', 'iid', 'hour', 'text', 'month', 'truth', 'A1_1', 'A2_1',
       'A3_1', 'age_30-39', 'age_<=18', 'age_>=40', 'sex_male', 'org_non-org',
       'weekday_Mon', 'weekday_Sat', 'weekday_Sun', 'weekday_Thu',
       'weekday_Tue', 'weekday_Wed', 'result_1'],
      dtype='object')

In [ ]:
data=data[['hour', 'month', 'text', 'truth', 'A1_1', 'A2_1', 'A3_1',
       'age_30-39', 'age_<=18', 'age_>=40', 'sex_male', 'org_non-org',
       'weekday_Mon', 'weekday_Sat', 'weekday_Sun', 'weekday_Thu',
       'weekday_Tue', 'weekday_Wed', 'result_1']]

In [ ]:
X = data.drop('truth', axis=1)

y = data['truth']

In [ ]:
from sklearn import preprocessing

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
dic = {"`":"'", "’":"'", "‘":"'", "´":"'","ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                       "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                       "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                       "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                       "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                       "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                       "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                       "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                       "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
                       "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
                       "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                       "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                       "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are",
                       "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have",
                       "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
                       "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have",
                       "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have",
                       "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is",
                       "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
                       "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would",
                       "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have",
                       "It's": 'It is', "Can't": 'Can not', "I've": 'I have', "\n":" ", "—":"", ".":"", "…":"", "!":"", ":":" ",
                       "-":" ","•":""}

#"😍":"lovely" 

In [ ]:
def preprocess_text(sen):

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    for i, j in dic.items():
        sentence = sentence.lower().replace(i, j)
    sentence = re.sub(r"http[s]?://t.co/[A-Za-z0-9]*"," ",sentence) #URLs

    return sentence

In [ ]:
X1_train = []
sentences = list(X_train["text"])
for sen in sentences:
    X1_train.append(preprocess_text(sen))

In [ ]:
X1_test = []
sentences = list(X_test["text"])
for sen in sentences:
    X1_test.append(preprocess_text(sen))

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X1_train)

X1_train = tokenizer.texts_to_sequences(X1_train)
X1_test = tokenizer.texts_to_sequences(X1_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 27

X1_train = pad_sequences(X1_train, padding='post', maxlen=maxlen)
X1_test = pad_sequences(X1_test, padding='post', maxlen=maxlen)

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

# glove_file = open('glove.twitter.27B.200d.txt', encoding="utf8")

# for line in glove_file:
#     records = line.split()
#     word = records[0]
#     vector_dimensions = asarray(records[1:], dtype='float32')
#     embeddings_dictionary[word] = vector_dimensions

# glove_file.close()

embedding_matrix = zeros((vocab_size, 128))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
X.shape

In [ ]:
X2_train = X_train[['month','hour','A1_1', 'A2_1', 'A3_1', 'age_30-39', 'age_<=18',
       'age_>=40', 'sex_male', 'org_non-org', 'weekday_Mon', 'weekday_Sat',
       'weekday_Sun', 'weekday_Thu', 'weekday_Tue', 'weekday_Wed']].values
X2_test = X_test[['month','hour',  'A1_1', 'A2_1', 'A3_1', 'age_30-39', 'age_<=18',
       'age_>=40', 'sex_male', 'org_non-org', 'weekday_Mon', 'weekday_Sat',
       'weekday_Sun', 'weekday_Thu', 'weekday_Tue', 'weekday_Wed']].values




In [ ]:
from keras.layers import Input
from keras.layers import Dropout, Conv1D, GlobalMaxPool1D, GRU, GlobalAvgPool1D
input_1 = Input(shape=(maxlen,))

input_2 = Input(shape=(16,))

In [ ]:
embedding_layer = Embedding(vocab_size, output_dim = 128, input_length = 27, weights=[embedding_matrix], trainable=False)(input_1)
#LSTM_Layer_1 = Bidirectional(LSTM(128, dropout=0.25, recurrent_dropout=0.25))(embedding_layer)
LSTM_Layer_1 = LSTM(128, dropout=0.25, recurrent_dropout=0.25)(embedding_layer)
LSTM_Layer_2 = Dense(64, activation='relu')(LSTM_Layer_1)
LSTM_Layer_3 = Dense(128, activation='relu')(LSTM_Layer_2)

In [ ]:
dense_layer_1 = Dense(128, activation='relu')(input_2)
dense_layer_2 = Dense(64, activation='relu')(dense_layer_1)

In [ ]:
from keras.layers import concatenate
from keras.models import Model
from keras.layers import Dropout, Conv1D, GlobalMaxPool1D, GRU, GlobalAvgPool1D

# concat_layer = concatenate([LSTM_Layer_2, dense_layer_2])
concat_layer = dense_layer_2
dense_layer_3 = Dense(128, activation='relu')(concat_layer)
output = Dense(2, activation='softmax')(dense_layer_3)
model = Model(inputs=[input_1, input_2], outputs=output)

In [ ]:
from keras import backend as K
import tensorflow as tf
from sklearn.metrics import roc_auc_score


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def auroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)




model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', f1_m,precision_m, recall_m, auroc])
print(model.summary())

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model_plot3.png', show_shapes=True, show_layer_names=True)

In [ ]:
history = model.fit(x=[X1_train, X2_train], y=y_train, batch_size=128, epochs=50, verbose=1, validation_split=0.2)

In [ ]:
loss, accuracy, f1_score, precision, recall, auroc  = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)



In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import fbeta_score, make_scorer
import sklearn.metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

predictions = model.predict([X1_test, X2_test])
y_pred = (predictions > 0.5)
matrix = sklearn.metrics.confusion_matrix( y_test.argmax(axis=1), y_pred.argmax(axis=1))

from sklearn.metrics import confusion_matrix
import numpy as np

#confusion = confusion_matrix(y_test, np.argmax(test_predictions,axis=1))

In [ ]:
matrix

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
precision, recall, _ = precision_recall_curve(y_test.argmax(axis=1), y_pred.argmax(axis=1))
auc_score = auc(recall, precision)

In [ ]:
recall

In [ ]:
auc_score

In [ ]:
y_test

-------------------------------------------


In [ ]:
from cleanlab.latent_estimation import estimate_cv_predicted_probabilities
import cleanlab

# Find the indices of label errors in 2 lines of code.
df_out = pd.read_csv("dfout.csv")
X = df_out.drop('label', 1).values
y=df_out['label']
s = np.array(y)

probabilities = estimate_cv_predicted_probabilities(
    X, 
    s, 
    clf=model,
)
psx = cleanlab.latent_estimation.estimate_cv_predicted_probabilities(
    X, s, clf=LogisticRegression(max_iter=1000, multi_class='auto', solver='lbfgs'))